In [1]:
!pip install ultralytics
!pip install roboflow
!pip install wandb
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [5]:
from roboflow import Roboflow
import os
import shutil

rf = Roboflow(api_key="TZCi2SCltMQkbaVAtOqT")

# List of datasets to download (workspace, dataset, version)
datasets_info = [
    ("hongmo", "sea-ezx3q", 44),
    ("kim-borma", "marine-debris-vb2qm", 16),
    ("myfyp", "new-taco", 3),
    ("anas-fd5bu", "-kg", 5),
    ("trash-detection-eka7g", "aeriel-trash-detection", 5),
    ("fyp-bfx3h", "yolov8-trash-detections", 6),
    ("taco-ihjgk", "yolov8-trash-detections-kgnug", 11),
]

"""
    ("trash-ai-27vay", "ultimate-rqkdd", 1),
    ("iqram", "trashdetectionv8", 2),
    ("smart-india-hackathon-2023", "garbage_best", 1),
    # New datasets (replace VERSION with the correct version number)
    ("kim-borma", "conttttt", 1),
    ("adamson-university-rkm6w", "manila-bay", 3),
    ("davids-workspace", "garbage-detection-mnj17", 1),
    ("majasworkspace", "p_eine_klasse", 1),
    ("ukm-wcn", "ml2-wcn-ukm", 23),
    ("vit-akped", "tacodetecron-2", 1),
    ("k2k", "imported-f38bi", 4),
    ("python-project-5oizd", "trash-detection-mzljs", 7),
    ("project-mqoot", "plastic-bag-detection", 1),
    ("trash-dataset-for-oriented-bounded-box", "trash-detection-1fjjc", 14),
    ("nam-nhat", "trash-dvdrr", 5),
    ("litter-beach-detection", "beach-garbage", 2),
    ("dechets-qlhbp", "tacoo-cj6rf", 1),
    ("cscsi218", "garbageclassification-47d7t", 1),
    ("hust-xz9js", "trashbot-wahhb", 1),
    ("furqan-sayyed-veuct", "taco-ppujr", 1),
    ("abhijeet-beedikar-pbc0x", "garbage-classification-taco", 1)
"""

dest_dir = os.path.abspath("datasets")
print(f"Destination directory: {dest_dir}")

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

for (workspace, dataset_name, version) in datasets_info:
    print(f"Processing {dataset_name}...")
    try:
        project = rf.workspace(workspace).project(dataset_name)
        print(f"Attempting to download version {version} of {dataset_name}")

        # Specify the download location
        download_dir = os.path.join(dest_dir, dataset_name)

        # Download the dataset to the specified location
        dataset = project.version(version).download("yolov11", location=download_dir)
        print(f"Successfully downloaded {dataset_name} to {dataset.location}")

        # List the contents of the downloaded directory
        if os.path.exists(dataset.location):
            print(f"Contents of {dataset.location}: {os.listdir(dataset.location)}")
        else:
            print(f"Warning: Downloaded directory not found at {dataset.location}")

    except Exception as e:
        print(f"Error processing {dataset_name}: {str(e)}")
    print("-" * 50)

print("Download process completed. Please check the following directory for your files:")
print(dest_dir)

Destination directory: /content/datasets
Processing sea-ezx3q...
loading Roboflow workspace...
loading Roboflow project...
Attempting to download version 44 of sea-ezx3q
Successfully downloaded sea-ezx3q to /content/datasets/sea-ezx3q
Contents of /content/datasets/sea-ezx3q: ['test', 'README.dataset.txt', 'valid', 'data.yaml', 'train', 'README.roboflow.txt']
--------------------------------------------------
Processing marine-debris-vb2qm...
loading Roboflow workspace...
loading Roboflow project...
Attempting to download version 16 of marine-debris-vb2qm
Successfully downloaded marine-debris-vb2qm to /content/datasets/marine-debris-vb2qm
Contents of /content/datasets/marine-debris-vb2qm: ['test', 'README.dataset.txt', 'valid', 'data.yaml', 'train', 'README.roboflow.txt']
--------------------------------------------------
Processing new-taco...
loading Roboflow workspace...
loading Roboflow project...
Attempting to download version 3 of new-taco
Successfully downloaded new-taco to /cont

ValueError: too many values to unpack (expected 3)

In [6]:
import os
import shutil
import yaml
import glob

def merge_datasets(datasets_dir, combined_dataset_dir):
    # Create directories for the combined dataset
    os.makedirs(combined_dataset_dir, exist_ok=True)
    splits = ['train', 'valid', 'test']
    for split in splits:
        os.makedirs(os.path.join(combined_dataset_dir, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(combined_dataset_dir, 'labels', split), exist_ok=True)

    combined_classes = []
    class_mappings = {}

    # First pass: Collect all class names and build combined_classes list
    for dataset_name in os.listdir(datasets_dir):
        dataset_path = os.path.join(datasets_dir, dataset_name)
        if not os.path.isdir(dataset_path):
            continue

        data_yaml_path = os.path.join(dataset_path, 'data.yaml')
        if not os.path.exists(data_yaml_path):
            print(f"data.yaml not found in {dataset_path}. Skipping this dataset.")
            continue

        with open(data_yaml_path, 'r') as f:
            data_yaml = yaml.safe_load(f)

        dataset_classes = data_yaml.get('names', [])
        if not dataset_classes:
            print(f"No 'names' field found in {data_yaml_path}. Skipping this dataset.")
            continue

        # Update combined_classes and create class mappings for this dataset
        mapping = {}
        for idx, class_name in enumerate(dataset_classes):
            if class_name not in combined_classes:
                combined_classes.append(class_name)
            mapping[idx] = combined_classes.index(class_name)
        class_mappings[dataset_name] = mapping

    print(f"Combined classes: {combined_classes}")

    # Second pass: Copy images and labels, remap class IDs
    for dataset_name in os.listdir(datasets_dir):
        dataset_path = os.path.join(datasets_dir, dataset_name)
        if not os.path.isdir(dataset_path):
            continue

        mapping = class_mappings.get(dataset_name)
        if mapping is None:
            continue  # Skip datasets that were not included in combined_classes

        for split in ['train', 'valid', 'test']:
            split_dir = os.path.join(dataset_path, split)
            if not os.path.exists(split_dir):
                print(f"Split '{split}' not found in {dataset_name}. Skipping.")
                continue

            # Check if images and labels are under 'images' and 'labels' directories
            images_dir = os.path.join(split_dir, 'images')
            labels_dir = os.path.join(split_dir, 'labels')
            if not os.path.exists(images_dir) or not os.path.exists(labels_dir):
                # Maybe images and labels are directly under split_dir
                images_dir = split_dir
                labels_dir = split_dir
                # Check if there are images and labels in split_dir
                image_files = glob.glob(os.path.join(images_dir, '*.jpg')) + \
                              glob.glob(os.path.join(images_dir, '*.jpeg')) + \
                              glob.glob(os.path.join(images_dir, '*.png')) + \
                              glob.glob(os.path.join(images_dir, '*.bmp')) + \
                              glob.glob(os.path.join(images_dir, '*.tif')) + \
                              glob.glob(os.path.join(images_dir, '*.tiff'))
                label_files = glob.glob(os.path.join(labels_dir, '*.txt'))
                if len(image_files) == 0:
                    print(f"No images found in {images_dir}. Skipping split '{split}' in dataset '{dataset_name}'.")
                    continue
            else:
                # Images and labels are under 'images' and 'labels' directories
                image_files = glob.glob(os.path.join(images_dir, '*.*'))
                label_files = glob.glob(os.path.join(labels_dir, '*.txt'))

            for image_file in image_files:
                image_filename = os.path.basename(image_file)
                image_name, image_ext = os.path.splitext(image_filename)

                # Corresponding label file
                label_file = os.path.join(labels_dir, f"{image_name}.txt")
                if not os.path.exists(label_file):
                    print(f"Label file {label_file} not found. Creating empty label.")
                    label_lines = []
                else:
                    with open(label_file, 'r') as lf:
                        label_lines = lf.readlines()

                # Remap class IDs in label file
                remapped_labels = []
                for line in label_lines:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        print(f"Invalid label format in {label_file}. Skipping line.")
                        continue
                    try:
                        class_id = int(parts[0])
                        new_class_id = mapping.get(class_id)
                        if new_class_id is None:
                            print(f"Class ID {class_id} not found in mapping for {dataset_name}. Skipping line.")
                            continue
                        parts[0] = str(new_class_id)
                        remapped_labels.append(' '.join(parts))
                    except ValueError:
                        print(f"Invalid class ID in {label_file}. Skipping line.")
                        continue

                # Copy image
                dest_image_name = f"{dataset_name}_{image_filename}"
                target_split = split  # Use the same split name
                dest_image_path = os.path.join(combined_dataset_dir, 'images', target_split, dest_image_name)
                shutil.copy(image_file, dest_image_path)

                # Write remapped label
                dest_label_name = f"{dataset_name}_{image_name}.txt"
                dest_label_path = os.path.join(combined_dataset_dir, 'labels', target_split, dest_label_name)
                with open(dest_label_path, 'w') as lf:
                    lf.write('\n'.join(remapped_labels))

    # Create combined data.yaml
    combined_data = {
        'path': os.path.abspath(combined_dataset_dir),
        'train': os.path.join('images', 'train'),
        'val': os.path.join('images', 'valid'),  # Use 'valid' as validation set
        'names': combined_classes
    }
    with open(os.path.join(combined_dataset_dir, 'data.yaml'), 'w') as f:
        yaml.dump(combined_data, f)

    print("Dataset merging complete.")

# Usage
datasets_dir = 'datasets'  # Directory containing individual datasets
combined_dataset_dir = 'combined_dataset'  # Directory to store the combined dataset

merge_datasets(datasets_dir, combined_dataset_dir)

Combined classes: ['Aluminium foil', 'Bottle', 'Bottle cap', 'Can', 'Carton', 'Cigarette', 'Glass bottle', 'Lid', 'Metal', 'Metal cap', 'Other litter', 'Other plastic', 'Paper', 'Plastic bag', 'Plastic buoy', 'Plastic vessels', 'Pop tab', 'Straw', 'Styrofoam cup', 'Styrofoam piece', 'Styrofoam_Buoy', 'Wrapper', 'Glass', 'Net', 'PET_Bottle', 'Plastic_Buoy', 'Plastic_Buoy_China', 'Plastic_ETC', 'Rope', 'Styrofoam_Box', 'Styrofoam_Piece', 'PET', 'Styrofoam', 'cardboard', 'fabrics', 'glass', 'metal', 'other', 'plastic', 'wood']
Dataset merging complete.


In [ ]:
import yaml
import os
from collections import defaultdict

def consolidate_classes(combined_dataset_dir):
    data_yaml_path = os.path.join(combined_dataset_dir, 'data.yaml')

    # Load the original data.yaml
    with open(data_yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    # Define class mappings (adjust this based on your specific classes)
    class_mappings = {
        'Glass': ['Glass', 'glass', 'Glass bottle', 'glass bottle'],
        'Metal': ['Metal', 'metal', 'Can', 'can', 'aluminum', 'Aluminium foil', 'Pop tab', 'pop tab'],
        'Plastic': ['Plastic', 'plastic', 'PET', 'PET_Bottle', 'Plastic_ETC', 'plastic bottle', 'plastic container',
                    'Plastic bag', 'plastic bag', 'Plastic_Buoy', 'Plastic_Buoy_China', 'Bottle', 'Plastic vessels'],
        'Styrofoam': ['Styrofoam', 'Styrofoam_Box', 'Styrofoam_Buoy', 'Styrofoam_Piece', 'styrofoam', 'Styrofoam cup'],
        'Paper': ['Paper', 'paper', 'cardboard', 'cardboard boxes and cartons', 'Carton'],
        'Rope': ['Rope', 'Net'],
        'Food Waste': ['food - others', 'Decomposable'],
        'Other': ['Other litter', 'other', 'Non-decomposable', 'Non-decomposable-', 'utensils and straw', 'Straw',
                  'fabrics', 'wood', 'Cigarette', 'Wrapper', 'Lid', 'cap or lid', 'Bottle cap', 'plastic bottle cap']
    }

    # Create reverse mapping
    reverse_mapping = {}
    for new_class, old_classes in class_mappings.items():
        for old_class in old_classes:
            reverse_mapping[old_class.lower()] = new_class

    # Function to update labels in a file
    def update_labels(file_path):
        with open(file_path, 'r') as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts:
                old_class = int(parts[0])
                if old_class < len(data['names']):
                    old_class_name = data['names'][old_class].lower()
                    new_class_name = reverse_mapping.get(old_class_name, 'Other')
                    new_class_id = list(class_mappings.keys()).index(new_class_name)
                    new_lines.append(f"{new_class_id} {' '.join(parts[1:])}\n")

        with open(file_path, 'w') as f:
            f.writelines(new_lines)

    # Update all label files
    for split in ['train', 'valid', 'test']:
        label_dir = os.path.join(combined_dataset_dir, 'labels', split)
        for filename in os.listdir(label_dir):
            if filename.endswith('.txt'):
                update_labels(os.path.join(label_dir, filename))

    # Update data.yaml with new classes
    data['names'] = list(class_mappings.keys())

    # Save the updated data.yaml
    with open(data_yaml_path, 'w') as f:
        yaml.dump(data, f)

    print(f"Updated data.yaml saved to {data_yaml_path}")
    print(f"New classes: {data['names']}")

# Usage
datasets_dir = 'datasets'  # Directory containing individual datasets
combined_dataset_dir = 'combined_dataset'  # Directory storing the combined dataset

# First, run your existing merge_datasets function
merge_datasets(datasets_dir, combined_dataset_dir)

# Then, run the class consolidation
consolidate_classes(combined_dataset_dir)

# Now you can proceed with your YOLO training using the consolidated dataset

In [ ]:
import os
import wandb
from wandb.integration.ultralytics import add_wandb_callback
import torch
from ultralytics import YOLO
from huggingface_hub import HfApi, Repository, login
import shutil
from datetime import datetime
import sys
from pathlib import Path

# Hugging Face settings
HF_REPO = "EduTech-YOLOv11"
HF_USERNAME = "shng2025"  # Your Hugging Face username

def ensure_hf_login():
    """Ensure user is logged in to Hugging Face"""
    try:
        api = HfApi()
        # This will raise an error if not logged in
        api.whoami()
        print("Already logged in to Hugging Face")
    except Exception:
        print("Please log in to Hugging Face")
        login()

def setup_hf_repo(branch_name):
    """Setup or connect to Hugging Face repository"""
    api = HfApi()
    repo_id = f"{HF_USERNAME}/{HF_REPO}"

    # Clone repository if it doesn't exist locally
    if not os.path.exists(HF_REPO):
        print(f"Cloning repository {repo_id}...")
        Repository(
            local_dir=HF_REPO,
            clone_from=repo_id,
        )

    repo = Repository(
        local_dir=HF_REPO,
    )

    # Create and checkout new branch or checkout existing branch
    try:
        repo.git_checkout(branch_name, create_branch_ok=True)
        print(f"Switched to branch: {branch_name}")
    except Exception as e:
        print(f"Error creating/checking out branch: {e}")
        sys.exit(1)

    return repo

def save_checkpoint(model, epoch, repo, branch_name):
    """Save checkpoint to Hugging Face repository"""
    checkpoint_dir = os.path.join(HF_REPO, "checkpoints", f"epoch_{epoch}")
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Save model state
    checkpoint_path = os.path.join(checkpoint_dir, f"model_epoch_{epoch}.pt")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.model.state_dict(),
        'optimizer_state_dict': model.optimizer.state_dict() if hasattr(model, 'optimizer') else None,
    }, checkpoint_path)

    # Also save to root directory for latest version
    latest_path = os.path.join(HF_REPO, "latest_model.pt")
    shutil.copy(checkpoint_path, latest_path)

    # Commit and push to Hugging Face
    repo.git_add(auto_lfs_track=True)
    repo.git_commit(f"Checkpoint at epoch {epoch}")
    repo.git_push(upstream="origin", branch=branch_name)
    print(f"Checkpoint saved and pushed to branch {branch_name}")

def get_available_branches():
    """Get list of available branches from Hugging Face repo"""
    api = HfApi()
    repo_id = f"{HF_USERNAME}/{HF_REPO}"
    try:
        branches = api.list_repo_refs(repo_id).branches
        return [b.name for b in branches]
    except Exception as e:
        print(f"Error fetching branches: {e}")
        return []

def find_latest_checkpoint(repo_path, branch_name):
    """Find the latest checkpoint in the repository"""
    checkpoint_dir = os.path.join(repo_path, "checkpoints")
    if not os.path.exists(checkpoint_dir):
        return None, 0

    epochs = []
    for d in os.listdir(checkpoint_dir):
        if d.startswith("epoch_"):
            try:
                epoch_num = int(d.split("_")[1])
                epochs.append(epoch_num)
            except ValueError:
                continue

    if not epochs:
        return None, 0

    latest_epoch = max(epochs)
    return os.path.join(checkpoint_dir, f"epoch_{latest_epoch}", f"model_epoch_{latest_epoch}.pt"), latest_epoch

def main():
    # Ensure logged in to Hugging Face
    ensure_hf_login()

    # Ask user whether to continue existing run or start new one
    print("\nAvailable options:")
    print("1. Start new training run")
    print("2. Continue existing run")
    choice = input("Enter your choice (1/2): ")

    if choice == "2":
        branches = get_available_branches()
        if not branches:
            print("No existing branches found. Starting new run.")
            choice = "1"
        else:
            print("\nAvailable branches:")
            for i, branch in enumerate(branches, 1):
                print(f"{i}. {branch}")
            branch_idx = int(input("\nSelect branch number: ")) - 1
            branch_name = branches[branch_idx]
            run_id = branch_name  # Use branch name as wandb run ID

    if choice == "1":
        # Generate new run name for both wandb and branch
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_id = f"yolov11_run_{timestamp}"
        branch_name = run_id

    # Initialize wandb
    wandb.init(
        project="Edutech",
        id=run_id if choice == "2" else None,
        resume="must" if choice == "2" else None,
        config={
            'epochs': 100,
            'batch_size': 16,
            'learning_rate': 0.001,
            'fp16': True,
            'compile': True,
        }
    )

    # Setup HuggingFace repository and branch
    repo = setup_hf_repo(branch_name)

    # Load model
    model = YOLO('yolo11x.pt')

    # Enable torch.compile if available
    if torch.__version__ >= "2" and wandb.config.compile:
        print("Compiling model with torch.compile()...")
        model.model = torch.compile(model.model)

    # Add WandB callback
    add_wandb_callback(model)

    # Load checkpoint if continuing existing run
    start_epoch = 0
    if choice == "2":
        checkpoint_path, start_epoch = find_latest_checkpoint(HF_REPO, branch_name)
        if checkpoint_path and os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path)
            model.model.load_state_dict(checkpoint['model_state_dict'])
            if checkpoint['optimizer_state_dict']:
                model.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            print(f"Resuming from epoch {start_epoch}")

    # Training loop
    for epoch in range(start_epoch, 100, 5):  # Train in 5-epoch intervals
        print(f"\nStarting training from epoch {epoch} to {epoch + 5}")
        model.train(
            data='combined_dataset/data.yaml',
            epochs=5,
            imgsz=640,
            batch=16,
            workers=8,
            exist_ok=True,
            device='0',
            project=wandb.run.project,
            verbose=True,
            amp=True,
            resume=True if epoch > start_epoch else False,
        )

        # Save checkpoint
        save_checkpoint(model, epoch + 5, repo, branch_name)
        print(f"Checkpoint saved for epoch {epoch + 5}")

    wandb.finish()

if __name__ == "__main__":
    main()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

                   all       6258      21792      0.524      0.498      0.469      0.359


Generating Visualizations for batch-1/1:   0%|          | 0/32 [00:00<?, ?it/s]

Generating Visualizations for batch-1/1:   0%|          | 0/32 [00:00<?, ?it/s]


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5      16.9G      1.092      1.258      1.218         96        640:   1%|          | 17/2006 [00:12<24:41,  1.34it/s]

In [ ]:
# Run inference using the best model
results = model.predict(
    source=os.path.join(combined_dataset_dir, 'images/val'),
    save=True,
    project='runs/predict',
    name='yolo11x_predict',
    exist_ok=True
)


In [ ]:

import matplotlib.pyplot as plt
import glob
import os

def visualize_predictions(result_dir):
    image_paths = glob.glob(os.path.join(result_dir, '*.jpg'))
    num_images = min(4, len(image_paths))

    plt.figure(figsize=(15, 12))
    for i, image_path in enumerate(image_paths[:num_images]):
        image = plt.imread(image_path)
        plt.subplot(2, 2, i + 1)
        plt.imshow(image)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

visualize_predictions('runs/predict/yolo11x_predict')
